In [121]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
df = pd.read_csv('./dws_dsc_customer_opportunity_monthly_kpi_1.csv')
op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
op.columns = [re.sub('\w+\.', '', i) for i in list(op.columns)]
df = df.fillna(0)
# df.columns
df['if_cooped'] = 0
df['if_cooped'] = df['if_cooped'].where(df['contract_signed_opportunity_num'] < 1, 1) # 今年有没合作。
cus_bf_21 = op[(op['createdon'] < '2021-11-01') & (op['milestone'] == 'Contract Signed')]['account_name_en'].unique()
cus_bf_21 = list(cus_bf_21)
cus_af_21 = list(df['customer_name'].unique())
df['old_cust'] = df['customer_name'].isin(cus_bf_21)
# df.to_csv('./data_down/dws_dsc_customer_opportunity_monthly_kpi_1.csv', index = None, encoding='utf_8_sig')


colors = sns.color_palette('rocket')[0:5]
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# plt1

In [111]:
def plot_pie(plt10, plt11,  bi_title, **kwargs):
    """
    成功, 总和, 
    """
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
    fig.add_trace(
        go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
        labels = ['succeed', 'fail']),
        1, 1)

    fig.add_trace(
        go.Pie(values = [plt10, plt11 - plt10], 
        labels = ['succeed', 'fail']),
        1, 2)
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_layout(title_text=bi_title,
    width=900,
    height=400,
    margin=dict(l=10, r=10, t=50, b=20),
    paper_bgcolor="LightSteelBlue",
)

    return fig

plt10 = sum(df['contract_signed_opportunity_num'])  # win
plt11 = sum(df['total_opportunity_num']) # 整体的商机成功率.a
# plot_pie(plt10, plt11,  "整体的商机本年成功率\t\n 成功数量")

In [113]:
# plt20 = df.query("old_cust == 1 & if_cooped == 1 & total_opportunity_num > 0").shape[0] 
# plt21 = df.query("old_cust == 1 & total_opportunity_num > 0").shape[0] 
# # 0.592375366568915 旧客户留存率。
# plot_pie(plt20, plt21, '老客户留存率 \n\t 老客户留存数')

plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['contract_signed_opportunity_num'].sum()
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['total_opportunity_num'].sum() 
# plot_pie(plt30, plt31,  "商机level: 老客本年商机成功率\t\n 成功数量")

In [263]:
# df.query("old_cust == 0 & total_opportunity_num > 0 & ")['total_opportunity_num'].sum() 

In [114]:
# [df.query("old_cust == 1")['total_opportunity_num'].sum(), df.query("old_cust == 1")['total_opportunity_num'].sum() / sum(df['total_opportunity_num'])]
plt32 = df.query("old_cust == 0 & total_opportunity_num > 0")['contract_signed_opportunity_num'].sum() 
plt33 = df.query("old_cust == 0 & total_opportunity_num > 0")['total_opportunity_num'].sum()  # xinke zong
 
# plot_pie(plt32, plt33, '商机level: 新客户本年商机成功率 \n\t 新客户本年商机成功数')

---
# 客户

In [115]:
# plt42 = len(set(np.intersect1d(cus_bf_21, cus_af_21)))
# plt43 = len(cus_af_21)

# plot_pie(plt42, plt43, '客户level :  本年有商机的老客占比 \n\t 老客户数')
# # """# [341, 717]  今年出现的过往的客户 vs 今年的所有"""

In [116]:
# plt40 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['customer_name'].nunique()
# plt41 = df.query("old_cust == 0 & total_opportunity_num > 0")['customer_name'].nunique()
# plot_pie(plt40, plt41, '客户level :  客户转化率 \n\t 客户转化数')


In [91]:
df[df['contract_signed_opportunity_num'] > 0]['customer_name'].nunique()
df[df['contract_signed_opportunity_num'] <= 0]['customer_name'].nunique()

479

--- 
试探

In [135]:
op = op[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  
op = op.fillna(0)
op = op[op['contract_term'] < 60]
# op['contract_value_log'] = np.log(op['contract_value'])
op = op[op['contract_term'] != 0]
opp = op

In [149]:
opp = op
mst_list = ['Contract Signed', 'Closed-Canceled', 'Closed-Lost']
sctr_list = ['Other', 0]
prodc_list = [0]

opp = opp[opp['milestone'].isin(mst_list)]
opp = opp[~opp['sector'].isin(sctr_list)]
opp = opp[~opp['product1'].isin(prodc_list)]

opp = opp[opp['contract_value'] != 0]
opp['contract_value_log'] = np.log(op['contract_value'])
px.scatter(opp, x = 'contract_term', y = 'contract_value_log',\
     color="sector", trendline = 'ols')


In [136]:
# import statsmodels.formula.api as smf
# smf.ols('contract_value ~ contract_term + product1', data = op).fit().summary()


C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



In [148]:
# 商机金额大小与转化率的关系。 c
# df = df[(df['milestone'] != 'Early Lead') & (df['milestone'] != 'Potential Opportunity')]
def load_opp(df):
     df['win_or_loss'] = 0
     df['win_or_loss'] = df['win_or_loss'].where(df['milestone']  != 'Contract Signed', 1)
     df = df.fillna(0)
     # 删除有汉字的。
     df = df[~df['annual_average_gross_profit'].str.contains('[az]').fillna(False)]
     return df
opp = load_opp(opp)
smf.logit(formula = "win_or_loss ~ contract_value ", data = opp).fit().summary()

Optimization terminated successfully.
         Current function value: 0.675124
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            win_or_loss   No. Observations:                 1648
Model:                          Logit   Df Residuals:                     1646
Method:                           MLE   Df Model:                            1
Date:                Wed, 08 Dec 2021   Pseudo R-squ.:                0.001349
Time:                        17:49:16   Log-Likelihood:                -1112.6
converged:                       True   LL-Null:                       -1114.1
Covariance Type:            nonrobust   LLR p-value:                   0.08299
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.3957      0.052      7.544      0.000       0.293       0.498
contract_value  -1.61e-06   1.16e-06     -1.393      0.164   -3.88e-06    6.56e-07
==================================================================================
"""

In [143]:
# go.Figure(go.Sunburst(
#     labels=[ "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
#     parents=[  "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
#     values=[  14, 12, 10, 2, 6, 6, 4, 4],
# ))

---
# sap

(238, 46)